# Setup

## Imports

In [18]:
import Plotly from "tslab-plotly";
import * as tslab from "tslab";

## Shared layout

In [19]:
const sharedLayout = {
    width: 600,
    height: 400,
    font: {
        size: 15,
    },
    yaxis: {
        automargin: true,
    },
    xaxis: {
        automargin: true,
    },
    margin: {
        l: 4,
        r: 4,
        b: 4,
        t: 50, // for title
        pad: 0, // between axis and plot
    },
};

# Plotly demo

In [20]:
(() => {
    let trace1: Plotly.Data = {
        x: ['giraffes', 'orangutans', 'monkeys'],
        y: [20, 14, 23],
        name: 'Zoo 1',
        type: 'bar',
    };

    let trace2: Plotly.Data = {
        x: ['giraffes', 'orangutans', 'monkeys'],
        y: [12, 18, 29],
        name: 'Zoo 2',
        type: 'bar',
    };

    let trace3: Plotly.Data = {
        x: ['penguins', 'crocodiles', 'swans'],
        y: [16, 10, 28],
        name: 'Zoo 3',
        type: 'bar',

        xaxis: 'x2',
        yaxis: 'y2',
    };

    let trace4: Plotly.Data = {
        x: ['penguins', 'crocodiles', 'swans'],
        y: [24, 12, 16],
        name: 'Zoo 4',
        type: 'bar',

        xaxis: 'x2',
        yaxis: 'y2',
    };

    let data: Plotly.Data[] = [trace1, trace2, trace3, trace4];
    let layout: Partial<Plotly.Layout> = {
        ...sharedLayout,

        grid: { rows: 1, columns: 2, pattern: 'independent' },
        title: 'Animals in different zoos',
        barmode: 'group',
        annotations: [
            // subplot titles (hack)
            {
                text: 'First subplot',
                font: {
                    size: 16,
                },
                align: 'center',
                x: 0.13, //position in x domain
                y: 1.1, //position in y domain
                xref: 'paper',
                yref: 'paper',
                showarrow: false,
            },
            {
                text: 'Second subplot',
                font: {
                    size: 16,
                },
                align: 'center',
                x: 0.9, //position in x domain
                y: 1.1, // position in y domain
                xref: 'paper',
                yref: 'paper',
                showarrow: false,
            },
        ],
    };
    Plotly.newPlot(tslab, data, layout);
})();


# Loading report collections

In [21]:
import fs from 'fs';
import unpackUnityJson from './util/unpackUnityJson';
import loadReportCollection from './util/loadReportCollection';
import path from 'path';

const reportCollections = (() => {
    const reportDir = './Reports/';
    const reportCollections = {};
    for (const reportFileName of fs.readdirSync(reportDir)) {
        const reportPath = path.join(reportDir, reportFileName);

        let reportName = reportFileName;
        const reportNameParts = reportFileName.split('.');
        reportNameParts.splice(-1, 1);
        reportName = reportNameParts.join('');

        reportCollections[reportName] = loadReportCollection(
            unpackUnityJson(JSON.parse(fs.readFileSync(reportPath, 'utf-8')))
        ).reports;
    }

    return reportCollections;
})();

## Report format

In [28]:
(() => {
    const keys = Object.keys(reportCollections);
    console.dir({
        reports: keys,
        launchParameters: reportCollections[keys[0]][0].launchParameters,
    });
})();


{
  reports: [
    'Empty cluster randomization',
    'Random swap (1KM) vs Random swap (2KM)',
    'Scaling vs subsampling',
    'Scanline jitter',
    'Staggered jitter',
    'Subsampling'
  ],
  launchParameters: {
    dispatcherParameters: {},
    videoName: '2',
    numIterations: 3,
    workingTextureSize: 8,
    numClusters: 6,
    jitterSize: 1,
    staggeredJitter: false,
    doDownscale: false,
    algorithm: 'KM',
    doRandomizeEmptyClusters: false
  }
}


# Plots

## Subsampling

In [23]:
(() => {
    const subsamplingReports = reportCollections['Subsampling'];

    const textureSizeOptions = Array.from(
        new Set(
            subsamplingReports.map(
                (report) => report.launchParameters.workingTextureSize
            )
        )
    );

    const videoNameOptions = Array.from(
        new Set(
            subsamplingReports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = subsamplingReports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = textureSizeOptions.map(
                (textureSize) => {
                    const subsetTextureSize = subsetVideo
                        .filter(
                            (report) =>
                                report.launchParameters.workingTextureSize ==
                                textureSize
                        );

                    return {
                        y: subsetTextureSize.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetTextureSize.map(
                            (report) => `${report.launchParameters.numClusters}`
                        ),

                        name: `${textureSize}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'number of clusters',
                    type: 'category',
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: 'texture size',
                    },
                },

                margin: {
                    l: 80,
                    r: 4,
                    b: 40,
                    t: 50, // for title
                    pad: 0, // between axis and plot
                },

                title: `Subsampling (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();


## Subsample vs Downscale

In [24]:
(() => {
    const downscalingReports = reportCollections['Scaling vs subsampling'];

    const doDownscaleOptions = Array.from(
        new Set(
            downscalingReports.map(
                (report) => report.launchParameters.doDownscale
            )
        )
    );

    const videoNameOptions = Array.from(
        new Set(
            downscalingReports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = downscalingReports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = doDownscaleOptions.map(
                (doDownscale) => {
                    const subsetDoDownscale = subsetVideo.filter(
                        (report) =>
                            report.launchParameters.doDownscale == doDownscale
                    );

                    return {
                        y: subsetDoDownscale.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetDoDownscale.map(
                            (report) =>
                                `${report.launchParameters.workingTextureSize}`
                        ),

                        name: `${doDownscale ? 'downscale' : 'subsample'}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                margin: {
                    l: 80,
                    r: 4,
                    b: 40,
                    t: 50, // for title
                    pad: 0, // between axis and plot
                },

                title: `Downscaling vs subsampling (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();


## Jitter

### Scanline jitter

In [25]:
(() => {
    const scanlineJitterReports = reportCollections['Scanline jitter'];

    const jitterSizeOptions: number[] = Array.from(
        new Set(
            scanlineJitterReports.map((report) =>
                Number.parseInt(report.launchParameters.jitterSize)
            )
        )
    );
    jitterSizeOptions.sort((a, b) => a-b);

    const videoNameOptions = Array.from(
        new Set(
            scanlineJitterReports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = scanlineJitterReports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = jitterSizeOptions.map(
                (jitterSize) => {
                    const subsetJitterSize = subsetVideo.filter(
                        (report) =>
                            report.launchParameters.jitterSize == jitterSize
                    );

                    return {
                        y: subsetJitterSize.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetJitterSize.map(
                            (report) =>
                                `${report.launchParameters.workingTextureSize}`
                        ),

                        name: `${jitterSize}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: "jitter size"
                    }
                },

                margin: {
                    l: 80,
                    r: 4,
                    b: 40,
                    t: 50, // for title
                    pad: 0, // between axis and plot
                },

                title: `Scanline jitter (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();

### Staggered jitter

In [26]:
(() => {
    const scanlineJitterReports = reportCollections['Staggered jitter'];

    const jitterSizeOptions: number[] = Array.from(
        new Set(
            scanlineJitterReports.map((report) =>
                Number.parseInt(report.launchParameters.jitterSize)
            )
        )
    );
    jitterSizeOptions.sort((a, b) => a-b);

    const videoNameOptions = Array.from(
        new Set(
            scanlineJitterReports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = scanlineJitterReports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = jitterSizeOptions.map(
                (jitterSize) => {
                    const subsetJitterSize = subsetVideo.filter(
                        (report) =>
                            report.launchParameters.jitterSize == jitterSize
                    );

                    return {
                        y: subsetJitterSize.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetJitterSize.map(
                            (report) =>
                                `${report.launchParameters.workingTextureSize}`
                        ),

                        name: `${jitterSize}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: "jitter size"
                    }
                },

                margin: {
                    l: 80,
                    r: 4,
                    b: 40,
                    t: 50, // for title
                    pad: 0, // between axis and plot
                },

                title: `Staggered jitter (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();

### Difference

In [27]:
import { json } from 'stream/consumers';

(() => {
    const scanlineJitterReports = reportCollections['Scanline jitter'];
    const staggeredJitterReports = reportCollections['Staggered jitter'];

    const reportPairs = scanlineJitterReports.map((scanlineJitterReport) => {
        const launchParamsString = JSON.stringify(
            scanlineJitterReport.launchParameters
        );

        const staggeredJitterReport = staggeredJitterReports.find(
            (report) =>
                JSON.stringify(report.launchParameters) == launchParamsString
        );

        return {
            scanlineJitterReport: scanlineJitterReport,
            staggeredJitterReport: staggeredJitterReport,
            launchParameters: scanlineJitterReport.launchParameters,
        };
    });

    const jitterSizeOptions: number[] = Array.from(
        new Set(
            scanlineJitterReports.map((report) =>
                Number.parseInt(report.launchParameters.jitterSize)
            )
        )
    );
    jitterSizeOptions.sort((a, b) => a-b);

    const videoNameOptions = Array.from(
        new Set(
            scanlineJitterReports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = reportPairs.filter(
                (reportPair) =>
                    reportPair.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = jitterSizeOptions.map((jitterSize) => {
                const subsetJitterSize = subsetVideo.filter(
                    (reportPair) =>
                        reportPair.launchParameters.jitterSize == jitterSize
                );

                return {
                    y: subsetJitterSize.map(
                        (reportPair) =>
                            reportPair.scanlineJitterReport.measurement
                                .aggregated[aggregatedStatName] -
                            reportPair.staggeredJitterReport.measurement
                                .aggregated[aggregatedStatName]
                    ),

                    x: subsetJitterSize.map(
                        (reportPair) =>
                            `${reportPair.launchParameters.workingTextureSize}`
                    ),

                    name: `${jitterSize}`,
                    type: 'bar',
                };
            });

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                },

                yaxis: {
                    title: {
                        text: `Δ ${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: 'jitter size',
                    },
                },

                margin: {
                    l: 80,
                    r: 4,
                    b: 40,
                    t: 50, // for title
                    pad: 0, // between axis and plot
                },

                title: `Scanline jitter minus staggered jitter (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();


## Empty cluster randomization

In [31]:
(() => {
    const scanlineJitterReports = reportCollections['Empty cluster randomization'];

    const videoNameOptions = Array.from(
        new Set(
            scanlineJitterReports.map(
                (report) => report.launchParameters.videoName
            )
        )
    ).sort();

    for (const aggregatedStatName of ['mean', 'peak']) {
        for (const videoName of videoNameOptions) {
            const subsetVideo = scanlineJitterReports.filter(
                (report) => report.launchParameters.videoName == videoName
            );

            const data: Plotly.Data[] = [true, false].map(
                (doRandomizeEmptyClusters) => {
                    const subsetJitterSize = subsetVideo.filter(
                        (report) =>
                            report.launchParameters.doRandomizeEmptyClusters == doRandomizeEmptyClusters
                    );

                    return {
                        y: subsetJitterSize.map(
                            (report) =>
                                report.measurement.aggregated[
                                    aggregatedStatName
                                ]
                        ),

                        x: subsetJitterSize.map(
                            (report) =>
                                `${report.launchParameters.workingTextureSize}`
                        ),

                        name: `${doRandomizeEmptyClusters ? "randomized" : "unchanged"}`,
                        type: 'bar',
                    };
                }
            );

            // todo fix definitions Plotly.Layout
            const layout: any /*Partial<Plotly.Layout>*/ = {
                ...sharedLayout,

                xaxis: {
                    title: 'working texture size',
                    type: 'category',
                },

                yaxis: {
                    title: {
                        text: `${aggregatedStatName} variance`,
                        standoff: 4,
                    },
                    automargin: true,
                },

                legend: {
                    title: {
                        text: "empty clusters"
                    }
                },

                margin: {
                    l: 80,
                    r: 4,
                    b: 40,
                    t: 50, // for title
                    pad: 0, // between axis and plot
                },

                title: `Empty cluster randomization (video ${videoName})`,
            };

            Plotly.newPlot(tslab, data, layout);
            tslab.display.html('<br>');
        }
    }
})();